In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
from tqdm.auto import tqdm
from urllib.request import urlretrieve
from zipfile import ZipFile
 
 
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import glob
import sentencepiece as smp

In [ ]:
!nvidia-smi 

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')


In [26]:

# def process_local_folder(folder_path):
#     print(f"Processing files in folder: {folder_path}", end="")

#     if os.path.exists(folder_path):
#         try:
#             # Process the files within the folder
#             for file_name in os.listdir(folder_path):
#                 file_path = os.path.join(folder_path, file_name)
#                 # Add your processing logic here
#                 print(f"\nProcessing file: {file_path}")
            
#             print("\nDone")

#         except Exception as e:
#             print("\nError processing files.", e)
#     else:
#         print("\nFolder does not exist.")

# local_folder_path = r"C:\Users\oyku_\Desktop\images"

# process_local_folder(local_folder_path)


In [46]:
def download_and_unzip(url, save_path):
    print(f"Downloading and extracting assets....", end="")
 
 
    # Downloading zip file using urllib package.
    urlretrieve(url, save_path)
 
 
    try:
        # Extracting zip file using the zipfile package.
        with ZipFile(save_path) as z:
            # Extract ZIP file contents in the same directory.
            z.extractall(os.path.split(save_path)[0])
 
 
        print("Done")
 
 
    except Exception as e:
        print("\nInvalid file.", e)
 
URL = r"https://drive.google.com/drive/folders/110RqpECTHgyaqtWDF7QMymZl04vT8n0K?usp=drive_link"
asset_zip_path = os.path.join(os.getcwd(), "recipe_pics.zip")
# Download if assest ZIP does not exists.
if not os.path.exists(asset_zip_path):
    download_and_unzip(URL, asset_zip_path)

In [47]:
def read_image(image_path):
    """
    :param image_path: String, path to the input image.


    Returns:
        image: PIL Image.
    """
    image = Image.open(image_path).convert('RGB')
    return image

In [48]:
def ocr(image, processor, model):
    """
    :param image: PIL Image.
    :param processor: Huggingface OCR processor.
    :param model: Huggingface OCR model.


    Returns:
        generated_text: the OCR'd text string.
    """
    # We can directly perform OCR on cropped images.
    pixel_values = processor(image, return_tensors='pt').pixel_values.to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [53]:
def eval_new_data(data_path=None, num_samples=8, model=None):
    image_paths = glob.glob(data_path)
    for i, image_path in tqdm(enumerate(image_paths), total=len(image_paths)):
        if i == num_samples:
            break
        image = read_image(image_path)
        text = ocr(image, processor, model)
        plt.figure(figsize=(7, 4))
        plt.imshow(image)
        plt.title(text)
        plt.axis('off')
        plt.show()

In [14]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-printed')
model = VisionEncoderDecoderModel.from_pretrained(
    'microsoft/trocr-small-printed'
).to(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
print(torch.cuda.is_available())  # True döndermeli
print(torch.version.cuda)  


True
12.4


In [55]:
eval_new_data(
    data_path=os.path.join('recipe_pics','recipes', '*jpg'),
    num_samples=8,
    model=model
)

0it [00:00, ?it/s]

In [ ]:
# processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
# model = VisionEncoderDecoderModel.from_pretrained(

#     'microsoft/trocr-base-handwritten'
# ).to(device)

#FOR HANDWRİTTEN TEXT MUCH LARGER MODEL TAKES AWFUL LOT OF TIME TO RUN!!!!!